Todos: 
- Encoding: use VSM* to convert text from pdfs into vectors (unanswered: how much of the document can we feed before word2vec explodes?) *-- realized word2vec isn't what you use for document to vector, it'd need to be a vector space model 
- Models: use a SOTA transformer model? (unanswered: do we need to fine tune or can we use a previously created model as is? If fine tuning, what is our training set / testing set and do we have enough data?)
- Search/Ranking: Not that difficult once the model can produce predictions (use softmax/logit scores as relevancy score, can naively search pdf for location of citation)

In [17]:
from PyPDF2 import PdfReader
from os import listdir
from os.path import isfile, join
from collections import defaultdict

def get_pdfs():
    source_pdfs = [] # single array of abstract text for sources
    cited_pdfs = defaultdict(lambda: defaultdict(lambda: []) # source paper --> {relevant: [], nonrelevant: []}
    initial_path = './data/'
    papers = ['paper1', 'paper2', 'paper3', 'paper4', 'paper5', 'paper6', 'paper7']
    for paper in papers:
        source_paper_path = join(initial_path, paper)
        pdfs = [join(source_paper_path, f) for f in listdir(source_paper_path) if isfile(join(source_paper_path, f))]
        source_pdfs.append(pdfs[0])
        relevance = ['/Cited/Relevant', '/Cited/Less Relevant']
        for rel in relevance:
            cited_paper_path = source_paper_path + rel
            cited_pdf_paths = [join(cited_paper_path, f) for f in listdir(cited_paper_path) if isfile(join(cited_paper_path, f))]
            cited_pdfs[paper][rel] = cited_pdfs
    return source_pdfs, cited_pdfs

source_pdf_paths, cited_pdf_paths = get_pdfs()
print(source_pdf_paths)
print(cited_pdf_paths)
# reader = PdfReader(pdfs[0])
# number_of_pages = len(reader.pages)
# page = reader.pages[0]
# text = page.extract_text()
# print(pdfs)


SyntaxError: invalid syntax (<ipython-input-17-11b1f293642f>, line 9)